#  This is a shared notebook for Project I - Group 3

## Data Cleaning

In [ ]:
#Dependencies
import os
import pandas as pd
import calendar
#List of all my dependencies in case of issues. Sometimes I have had to import individual functions even though they
#are supposed to be in a package. Version problem maybe? Anyway worst case turn these all on.
#import os
#import pandas as pd
#import calendar
import glob
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.graphics.factorplots import interaction_plot
import statsmodels.api as sm
from scipy.stats import levene
from scipy.stats import bartlett
from scipy.stats import linregress
import pingouin as pg
from functools import reduce
from pingouin import kruskal, read_dataset

In [ ]:
csvDir = os.path.join("Resources")
col_list = ["month_of_death","day_of_week_of_death","current_data_year",
                              "manner_of_death","358_cause_recode","sex","detail_age"]
masterDf = pd.DataFrame()
files = os.listdir(csvDir)
for file in files:
    if file.endswith(".csv"):
        curCsv = os.path.join(csvDir,file)
        with open(curCsv) as file:
            x = pd.read_csv(file, usecols=col_list)
            masterDf = pd.concat([masterDf,x],axis=0)

In [ ]:
#Filter to only deaths related to cars
car_death_data = masterDf[(masterDf["358_cause_recode"] >=385) & (masterDf["358_cause_recode"] <=398)]
car_death_data.head()

In [ ]:
#Clean up the dataframe so that it is readable
month_dict = {
      1:"January",
      2:"February",
      3:"March",
      4:"April",
      5:"May",
      6:"June",
      7:"July",
      8:"August",
      9:"September",
      10:"October",
      11:"November",
      12:"December"}
      
day_of_week_dict = {
      1:"Sunday",
      2:"Monday",
      3:"Tuesday",
      4:"Wednesday",
      5:"Thursday",
      6:"Friday",
      7:"Saturday",
      9:"Unknown"}

manner_of_death_dict = {
      1:"Accident",
      2:"Suicide",
      3:"Homicide",
      4:"Pending investigation",
      5:"Could not determine",
      6:"Self-Inflicted",
      7:"Natural"}
#       "Blank":"Not specified"}

cause_recode_dict = {
      385:" 385- Motor vehicle accidents",
      386:" 386- Pedestrian involved in collision with motor vehicle",
      387:" 387- Pedalcyclist involved in collision with motor vehicle",
      388:" 388- Motorcyclist involved in any accident except collision with railway train",
      389:" 389- Motor vehicle accident involving collision with railway train",
      390:" 390- Motorcyclist involved in collision with railway train",
      391:" 391- Other motor vehicle accident involving collision with railway train",
      392:" 392- Occupant of motor vehicle involved in collision with other (non- motorized) road vehicle, streetcar, animal or pedestrian",
      393:" 393- Occupant of car, pickup truck or van involved in collision with other motor vehicle",
      394:" 394- Occupant of heavy transport vehicle or bus involved in collision with other motor vehicle",
      395:" 395- Occupant of motor vehicle involved in non-collision accident",
      396:" 396- Occupant of special-use motor vehicle involved in any accident",
      397:" 397- Other and unspecified motor vehicle accidents",
      398:" 398- Streetcar accidents"}
    

clean_df = car_death_data.replace({"month_of_death": month_dict, 
                        "day_of_week_of_death": day_of_week_dict,
                        "manner_of_death": manner_of_death_dict,
                        "358_cause_recode": cause_recode_dict})

clean_df

In [ ]:
#  check quality of data
#  list all unique values in each columns

colNames = list(clean_df.columns)
for col in colNames:
    print(col)
    print(f"{clean_df[col].unique()}")
    print("----------")

In [ ]:
#  Clean up
#  Remove not logical data 
# ie: age of 999, day of week : Unknown, manner_of_death nan, need filter by Accident
finalDf = clean_df[clean_df["detail_age"] != 999]
finalDf = finalDf[finalDf["manner_of_death"] == "Accident"]
finalDf =  finalDf[finalDf["day_of_week_of_death"] != "Unknown"]

## Analysis

## Get census data and merge with finaldf

In [ ]:
#Import data
census_summary_df = pd.read_csv("nc-est2019-agesex_rearranged.csv")
census_summary_df = census_summary_df.rename(columns={'Age group': 'Age_group', 'Both sexes': 'Both_sexes', 'Year':'current_data_year'})
census_summary_df

In [ ]:
def label_age_group (row):
    if row['detail_age'] < 5 :
        return '1'
    elif row['detail_age'] < 10 :
        return '2'
    elif row['detail_age'] < 15 :
        return '3'
    elif row['detail_age'] < 20 :
        return '4'
    elif row['detail_age'] < 25 :
        return '5'
    elif row['detail_age'] < 30 :
        return '6'
    elif row['detail_age'] < 35 :
        return '7'
    elif row['detail_age'] < 40 :
        return '8'
    elif row['detail_age'] < 45 :
        return '9'
    elif row['detail_age'] < 50 :
        return '10'
    elif row['detail_age'] < 55 :
        return '11'
    elif row['detail_age'] < 60 :
        return '12'
    elif row['detail_age'] < 65 :
        return '13'
    elif row['detail_age'] < 70 :
        return '14'
    elif row['detail_age'] < 75 :
        return '15'
    elif row['detail_age'] < 80 :
        return '16'
    elif row['detail_age'] < 85 :
        return '17'    
    else:
        return '18'

finalDf['Age_group'] = finalDf.apply (lambda row: label_age_group(row), axis=1)
finalDf.sort_values(by=['current_data_year'])
#finalDf

In [ ]:
#census_summary_df.dtypes

In [ ]:
#Correct datatypes
finalDf["Age_group"] = pd.to_numeric(finalDf["Age_group"])
#finalDf.dtypes

In [ ]:
#merge files
finalDf_with_census = pd.merge(finalDf, census_summary_df, how='left', on=['Age_group', 'current_data_year'])
finalDf_with_census.sort_values(by=['current_data_year'])
finalDf_with_census

## Total deaths by day of week

In [ ]:
#Make boxplot and save to file
total_deaths_by_d_and_y = finalDf.groupby(['day_of_week_of_death', 'current_data_year']).size().reset_index(name='counts')
#total_deaths_by_d_and_y
ax = sns.boxplot(x="day_of_week_of_death", y='counts', data=total_deaths_by_d_and_y)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
ax.set_title("Total Motor Vehicle Deaths by Day of Week (2005-2015)")
ax.set(xlabel='Day of Week of Death', ylabel='Deaths')
plt.subplots_adjust(bottom=0.3)
#plt.savefig('Deaths_by_day_of_week.pdf')

In [ ]:
#Levene test for equality of variance (pingouin)
#(acceptable for non-normal distributions)
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.levene.html

levene_test = pg.homoscedasticity(total_deaths_by_d_and_y, dv='counts', group='day_of_week_of_death')
print(levene_test)

In [ ]:
#One-way Anova with pingouin
aov = pg.anova(data=total_deaths_by_d_and_y, dv='counts', between='day_of_week_of_death', detailed=True)
print(aov)

#Low p value suggests can reject the null hypothesis that the population means of groups are equal.

## Total deaths by age group

In [ ]:
#Make boxplot and save to file
total_deaths_by_age_group_and_y = finalDf_with_census.groupby(['Age_group', 'current_data_year']).size().reset_index(name='counts')
#total_deaths_by_age_group_and_y
ax = sns.boxplot(x="Age_group", y='counts', data=total_deaths_by_age_group_and_y)
ax.set_xticklabels(ax.get_xticklabels())
ax.set_title("Total Motor Vehicle Deaths by Age Group (2005-2015)")
ax.set(xlabel='Age Group (years)', ylabel='Deaths')
plt.xticks([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], 
           ['0-4', '5-9', '10-14','15-19','20-24','25-29','30-34','35-39','40-44',
            '45-49','50-54','55-59','60-64','65-69','70-74','75-79','80-84','85 and over'])
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='center')
plt.subplots_adjust(bottom=0.3)
#plt.savefig('Deaths_by_age_group.pdf')

In [ ]:
#Levene test for equality of variance (pingouin)

levene_test = pg.homoscedasticity(total_deaths_by_age_group_and_y, dv='counts', group='Age_group')
print(levene_test)

In [ ]:
#Variances not equal, ANOVA not recommended

#Kruskal-Wallis H-test: Are differences among groups significant?
#https://pingouin-stats.org/generated/pingouin.kruskal.html
Kruskal_Wallis_H_test = kruskal(data=total_deaths_by_age_group_and_y, dv='counts', between='Age_group')
print(Kruskal_Wallis_H_test)

#Low p value suggests can reject the null hypothesis that the population median of groups are equal.

## Can ANOVA test for differences among weekdays be run on age groups individually?

In [ ]:
#Count deaths by age group, year, and day

total_deaths_by_age_group_y_and_day = finalDf_with_census.groupby(['Age_group', 'current_data_year', 'day_of_week_of_death']).size().reset_index(name='counts')
#total_deaths_by_age_group_y_and_day

In [ ]:
#Make faceted boxplot and save to file
#https://stackoverflow.com/questions/43920341/python-seaborn-facetgrid-change-titles

total_deaths_by_age_group_y_and_day['Age range'] = total_deaths_by_age_group_y_and_day['Age_group'].replace({1:'0-4', 2:'5-9', 3:'10-14', 4:'15-19', 5:'20-24', 6:'25-29', 7:'30-34', 8:'35-39', 9:'40-44',10:'45-49',11:'50-54', 12:'55-59' , 13:'60-64', 14:'65-69', 15:'70-74', 16:'75-79', 17:'80-84', 18:'85 and over'})

ax = sns.catplot(x="day_of_week_of_death", y="counts",
                col="Age range", col_wrap=6,
                data=total_deaths_by_age_group_y_and_day, kind="box",
                height=4, aspect=.7);
ax.set_axis_labels("", "Deaths")
ax.set_xticklabels(rotation=45, horizontalalignment='right')
plt.subplots_adjust(top=0.92, bottom=0.08)
ax.fig.suptitle('Total Motor Vehicle Deaths by Age Group and Day of Week of Death (2005-2015)', fontsize=16) 
#plt.savefig('Deaths_by_age_group_and_day_of_week.pdf')

In [ ]:
#Get separate age groups

grouped_by_age_group = finalDf_with_census.groupby(finalDf_with_census.Age_group)

Age_group_1 = grouped_by_age_group.get_group(1)
Age_group_2 = grouped_by_age_group.get_group(2)
Age_group_3 = grouped_by_age_group.get_group(3)
Age_group_4 = grouped_by_age_group.get_group(4)
Age_group_5 = grouped_by_age_group.get_group(5)
Age_group_6 = grouped_by_age_group.get_group(6)
Age_group_7 = grouped_by_age_group.get_group(7)
Age_group_8 = grouped_by_age_group.get_group(8)
Age_group_9 = grouped_by_age_group.get_group(9)
Age_group_10 = grouped_by_age_group.get_group(10)
Age_group_11 = grouped_by_age_group.get_group(11)
Age_group_12 = grouped_by_age_group.get_group(12)
Age_group_13 = grouped_by_age_group.get_group(13)
Age_group_14 = grouped_by_age_group.get_group(14)
Age_group_15 = grouped_by_age_group.get_group(15)
Age_group_16 = grouped_by_age_group.get_group(16)
Age_group_17 = grouped_by_age_group.get_group(17)
Age_group_18 = grouped_by_age_group.get_group(18)

Age_group_1_df_by_y_and_day = Age_group_1.groupby(['day_of_week_of_death', 'current_data_year']).size().reset_index(name='counts')
Age_group_2_df_by_y_and_day = Age_group_2.groupby(['day_of_week_of_death', 'current_data_year']).size().reset_index(name='counts')
Age_group_3_df_by_y_and_day = Age_group_3.groupby(['day_of_week_of_death', 'current_data_year']).size().reset_index(name='counts')
Age_group_4_df_by_y_and_day = Age_group_4.groupby(['day_of_week_of_death', 'current_data_year']).size().reset_index(name='counts')
Age_group_5_df_by_y_and_day = Age_group_5.groupby(['day_of_week_of_death', 'current_data_year']).size().reset_index(name='counts')
Age_group_6_df_by_y_and_day = Age_group_6.groupby(['day_of_week_of_death', 'current_data_year']).size().reset_index(name='counts')
Age_group_7_df_by_y_and_day = Age_group_7.groupby(['day_of_week_of_death', 'current_data_year']).size().reset_index(name='counts')
Age_group_8_df_by_y_and_day = Age_group_8.groupby(['day_of_week_of_death', 'current_data_year']).size().reset_index(name='counts')
Age_group_9_df_by_y_and_day = Age_group_9.groupby(['day_of_week_of_death', 'current_data_year']).size().reset_index(name='counts')
Age_group_10_df_by_y_and_day = Age_group_10.groupby(['day_of_week_of_death', 'current_data_year']).size().reset_index(name='counts')
Age_group_11_df_by_y_and_day = Age_group_11.groupby(['day_of_week_of_death', 'current_data_year']).size().reset_index(name='counts')
Age_group_12_df_by_y_and_day = Age_group_12.groupby(['day_of_week_of_death', 'current_data_year']).size().reset_index(name='counts')
Age_group_13_df_by_y_and_day = Age_group_13.groupby(['day_of_week_of_death', 'current_data_year']).size().reset_index(name='counts')
Age_group_14_df_by_y_and_day = Age_group_14.groupby(['day_of_week_of_death', 'current_data_year']).size().reset_index(name='counts')
Age_group_15_df_by_y_and_day = Age_group_15.groupby(['day_of_week_of_death', 'current_data_year']).size().reset_index(name='counts')
Age_group_16_df_by_y_and_day = Age_group_16.groupby(['day_of_week_of_death', 'current_data_year']).size().reset_index(name='counts')
Age_group_17_df_by_y_and_day = Age_group_17.groupby(['day_of_week_of_death', 'current_data_year']).size().reset_index(name='counts')
Age_group_18_df_by_y_and_day = Age_group_18.groupby(['day_of_week_of_death', 'current_data_year']).size().reset_index(name='counts')

In [ ]:
#Levene test for equal variance by day within age groups
levene_test_day_group_1 = levene_test_day_group_1 = pg.homoscedasticity(Age_group_1_df_by_y_and_day, dv='counts', group="day_of_week_of_death")
print(levene_test_day_group_1)
levene_test_day_group_2 = levene_test_day_group_1 = pg.homoscedasticity(Age_group_2_df_by_y_and_day, dv='counts', group="day_of_week_of_death")
print(levene_test_day_group_2)
levene_test_day_group_3 = levene_test_day_group_3 = pg.homoscedasticity(Age_group_3_df_by_y_and_day, dv='counts', group="day_of_week_of_death")
print(levene_test_day_group_3)
levene_test_day_group_4 = levene_test_day_group_4 = pg.homoscedasticity(Age_group_4_df_by_y_and_day, dv='counts', group="day_of_week_of_death")
print(levene_test_day_group_4)
levene_test_day_group_5 = levene_test_day_group_5 = pg.homoscedasticity(Age_group_5_df_by_y_and_day, dv='counts', group="day_of_week_of_death")
print(levene_test_day_group_5)
levene_test_day_group_6 = levene_test_day_group_6 = pg.homoscedasticity(Age_group_6_df_by_y_and_day, dv='counts', group="day_of_week_of_death")
print(levene_test_day_group_6)
levene_test_day_group_7 = levene_test_day_group_7 = pg.homoscedasticity(Age_group_7_df_by_y_and_day, dv='counts', group="day_of_week_of_death")
print(levene_test_day_group_7)
levene_test_day_group_8 = levene_test_day_group_8 = pg.homoscedasticity(Age_group_8_df_by_y_and_day, dv='counts', group="day_of_week_of_death")
print(levene_test_day_group_8)
levene_test_day_group_9 = levene_test_day_group_9 = pg.homoscedasticity(Age_group_9_df_by_y_and_day, dv='counts', group="day_of_week_of_death")
print(levene_test_day_group_9)
levene_test_day_group_10 = levene_test_day_group_10 = pg.homoscedasticity(Age_group_10_df_by_y_and_day, dv='counts', group="day_of_week_of_death")
print(levene_test_day_group_10)
levene_test_day_group_11 = levene_test_day_group_11 = pg.homoscedasticity(Age_group_11_df_by_y_and_day, dv='counts', group="day_of_week_of_death")
print(levene_test_day_group_11)
levene_test_day_group_12 = levene_test_day_group_12 = pg.homoscedasticity(Age_group_12_df_by_y_and_day, dv='counts', group="day_of_week_of_death")
print(levene_test_day_group_12)
levene_test_day_group_13 = levene_test_day_group_13 = pg.homoscedasticity(Age_group_13_df_by_y_and_day, dv='counts', group="day_of_week_of_death")
print(levene_test_day_group_13)
levene_test_day_group_14 = levene_test_day_group_14 = pg.homoscedasticity(Age_group_14_df_by_y_and_day, dv='counts', group="day_of_week_of_death")
print(levene_test_day_group_14)
levene_test_day_group_15 = levene_test_day_group_15 = pg.homoscedasticity(Age_group_15_df_by_y_and_day, dv='counts', group="day_of_week_of_death")
print(levene_test_day_group_15)
levene_test_day_group_16 = levene_test_day_group_16 = pg.homoscedasticity(Age_group_16_df_by_y_and_day, dv='counts', group="day_of_week_of_death")
print(levene_test_day_group_16)
levene_test_day_group_17 = levene_test_day_group_17 = pg.homoscedasticity(Age_group_17_df_by_y_and_day, dv='counts', group="day_of_week_of_death")
print(levene_test_day_group_17)
levene_test_day_group_18 = levene_test_day_group_18 = pg.homoscedasticity(Age_group_18_df_by_y_and_day, dv='counts', group="day_of_week_of_death")
print(levene_test_day_group_18)

#Yes, all variances are now equal, good to go with ANOVA

In [ ]:
#One-way ANOVA with pingouin
aov_day_group_1 = pg.anova(data = Age_group_1_df_by_y_and_day, dv='counts', between="day_of_week_of_death", detailed=True)
print(aov_day_group_1)
aov_day_group_2 = pg.anova(data = Age_group_2_df_by_y_and_day, dv='counts', between="day_of_week_of_death", detailed=True)
print(aov_day_group_2)
aov_day_group_3 = pg.anova(data = Age_group_3_df_by_y_and_day, dv='counts', between="day_of_week_of_death", detailed=True)
print(aov_day_group_3)
aov_day_group_4 = pg.anova(data = Age_group_4_df_by_y_and_day, dv='counts', between="day_of_week_of_death", detailed=True)
print(aov_day_group_4)
aov_day_group_5 = pg.anova(data = Age_group_5_df_by_y_and_day, dv='counts', between="day_of_week_of_death", detailed=True)
print(aov_day_group_5)
aov_day_group_6 = pg.anova(data = Age_group_6_df_by_y_and_day, dv='counts', between="day_of_week_of_death", detailed=True)
print(aov_day_group_6)
aov_day_group_7 = pg.anova(data = Age_group_7_df_by_y_and_day, dv='counts', between="day_of_week_of_death", detailed=True)
print(aov_day_group_7)
aov_day_group_8 = pg.anova(data = Age_group_8_df_by_y_and_day, dv='counts', between="day_of_week_of_death", detailed=True)
print(aov_day_group_8)
aov_day_group_9 = pg.anova(data = Age_group_9_df_by_y_and_day, dv='counts', between="day_of_week_of_death", detailed=True)
print(aov_day_group_9)
aov_day_group_10 = pg.anova(data = Age_group_10_df_by_y_and_day, dv='counts', between="day_of_week_of_death", detailed=True)
print(aov_day_group_10)
aov_day_group_11 = pg.anova(data = Age_group_11_df_by_y_and_day, dv='counts', between="day_of_week_of_death", detailed=True)
print(aov_day_group_11)
aov_day_group_12 = pg.anova(data = Age_group_12_df_by_y_and_day, dv='counts', between="day_of_week_of_death", detailed=True)
print(aov_day_group_12)
aov_day_group_13 = pg.anova(data = Age_group_13_df_by_y_and_day, dv='counts', between="day_of_week_of_death", detailed=True)
print(aov_day_group_13)
aov_day_group_14 = pg.anova(data = Age_group_14_df_by_y_and_day, dv='counts', between="day_of_week_of_death", detailed=True)
print(aov_day_group_14)
aov_day_group_15 = pg.anova(data = Age_group_15_df_by_y_and_day, dv='counts', between="day_of_week_of_death", detailed=True)
print(aov_day_group_15)
aov_day_group_16 = pg.anova(data = Age_group_16_df_by_y_and_day, dv='counts', between="day_of_week_of_death", detailed=True)
print(aov_day_group_16)
aov_day_group_17 = pg.anova(data = Age_group_17_df_by_y_and_day, dv='counts', between="day_of_week_of_death", detailed=True)
print(aov_day_group_17)
aov_day_group_18 = pg.anova(data = Age_group_18_df_by_y_and_day, dv='counts', between="day_of_week_of_death", detailed=True)
print(aov_day_group_18)

#All age groups but 14 (65-69) show a significant effect of day of week


In [ ]:
# FDR-corrected post hocs with Hedges'g effect size
posthoc = pg.pairwise_ttests(data=total_deaths_by_age_group_y_and_day, dv='counts', within='day_of_week_of_death', subject='Age_group',
                             parametric=True, padjust='fdr_bh', effsize='hedges')

# Pretty printing of table
pg.print_table(posthoc, floatfmt='.3f')

#For complete dataset, only Monday/Thursday and Tuesday/Wednesday show no significant difference in deaths

In [ ]:
# FDR-corrected post hocs with Hedges'g effect size
posthoc = pg.pairwise_ttests(data=total_deaths_by_age_group_y_and_day, dv='counts', within='Age_group', subject='day_of_week_of_death',
                             parametric=True, padjust='fdr_bh', effsize='hedges')

# Pretty printing of table
pg.print_table(posthoc, floatfmt='.3f')

#Grouped by age, all pairings were significantly different except: 4/6, 7/9, 8/12, 10/11, 17/18

## Compare total deaths and census data

In [ ]:
#Plot total motor vehicle deaths (and export to file)
ax = sns.countplot(hue="current_data_year", x="Age_group", data=finalDf)
ax.legend(bbox_to_anchor=(1.05, 1), loc=2,borderaxespad=0.)
ax.set_title("Total Motor Vehicle Deaths by Age Group and Year (2005-2015)", y=1.05)
plt.xticks([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], 
           ['0-4', '5-9', '10-14','15-19','20-24','25-29','30-34','35-39','40-44',
            '45-49','50-54','55-59','60-64','65-69','70-74','75-79','80-84','85 and over'])
sns.set(font_scale=1) 
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='center')
ax.set(xlabel='Age Group', ylabel='Deaths')

In [ ]:
#Plot population data (and export to file)
ax = sns.boxplot(x = 'Age_group', y = 'Both_sexes', data = census_summary_df)
ax.set_title("US Population by Age Group (2010-2019): Both sexes", y=1.05)
plt.xticks([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], 
           ['0-4', '5-9', '10-14','15-19','20-24','25-29','30-34','35-39','40-44',
            '45-49','50-54','55-59','60-64','65-69','70-74','75-79','80-84','85 and over'])
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='center')
ax.set(xlabel='Age Group', ylabel='Population')
ax.set(ylim=(0, 2.4e7))
plt.subplots_adjust(bottom=0.28)
sns.set(font_scale=1) 
#plt.savefig('US_pop_by_age_group.pdf')

In [ ]:
#Scatterplot of population vs. motor vehicle deaths, colored by age group

#Get counts and combine files
#https://stackoverflow.com/questions/17679089/pandas-dataframe-groupby-two-columns-and-get-counts
finalDf_with_census['COUNTER'] =1 
grouped = finalDf_with_census.groupby(['Age_group', 'current_data_year'])['COUNTER'].sum().reset_index(name ='total_accidents')
census_short_summary_df = pd.read_csv("nc-est2019-agesex_rearranged.csv")
census_short_summary_df.rename(columns={'Age group':'Age_group'}, inplace=True)
census_short_summary_df.rename(columns={'Year':'current_data_year'}, inplace=True)
census_short_summary_df.rename(columns={'Both sexes':'Both_sexes'}, inplace=True)
census_vs_total_accidents_df = pd.merge(grouped, census_short_summary_df)
#census_vs_total_accidents_df

In [ ]:
#Function for plotting linear regression :)

def plt_lin_reg(x_values, regress_values, line_eq, rvalue):
    plt.plot(x_values,regress_values,"k-")
    plt.text(0.2, 0.8, line_eq, fontsize=12, transform=plt.gcf().transFigure)
    r2 = (rvalue**2).round(decimals=2)
    r2_text = (f"r^2 = {r2}")
    plt.text(0.2, 0.75, r2_text, fontsize=12, transform=plt.gcf().transFigure)

In [ ]:
#Motor vehicle deaths vs. population size by age group: plot and save to file

#sns.palplot(sns.color_palette("muted"))
ax = sns.scatterplot(x='Both_sexes', y='total_accidents', hue='Age_group', palette=("muted"), data=census_vs_total_accidents_df)
ax.legend(bbox_to_anchor=(1.05, 1), loc=2,borderaxespad=0.)
new_labels = ['_', '0-4', '5-9', '10-14','15-19','20-24','25-29','30-34','35-39','40-44','45-49','50-54','55-59','60-64','65-69','70-74','75-79','80-84','85 and over']
for t, l in zip(ax.legend_.texts, new_labels): t.set_text(l)
ax.set(xlabel='Population', ylabel='Motor vehicle deaths')
ax.set_title('Motor Vehicle Deaths vs. Population by Age Group (2010-2015)', fontsize=14, y=1.05) 

#Calculate linear regression
y_values = census_vs_total_accidents_df['total_accidents']
x_values = census_vs_total_accidents_df['Both_sexes']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

#Plot and annotate linear regression
plt_lin_reg(x_values, regress_values, line_eq, rvalue)

#Save plot
#plt.savefig('Deaths vs. pop.pdf', bbox_inches='tight')
